In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("").getOrCreate()
#from pyspark.python.pyspark.shell import spark

your 131072x1 screen size is bogus. expect trouble
23/04/25 13:12:27 WARN Utils: Your hostname, WeebMachine resolves to a loopback address: 127.0.1.1; using 172.30.248.116 instead (on interface eth0)
23/04/25 13:12:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/25 13:12:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.load("Absenteeism_at_work.csv", format="csv", header=True, delimiter=",")

data = data.withColumn("MOA", data["Month of absence"] - 0).withColumn("label", data['Transportation expense'] - 0). \
    withColumn("ROA", data["Reason for absence"] - 0). \
    withColumn("distance", data["Distance from Residence to Work"] - 0). \
    withColumn("BMI", data["Body mass index"] - 0)
data.show(5)

23/04/25 13:12:37 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+---+-----+----+--------+----+
| ID|Reason for absence|Month of absence|Day of the week|Seasons|Transportation expense|Distance from Residence to Work|Service time|Age|Work load Average/day |Hit target|Disciplinary_failure|Education|Son|Social drinker|Social smoker|Pet|Weight|Height|Body mass index|Absenteeism_time_in_hours|MOA|label| ROA|distance| BMI|
+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+---+-----+----+--------+----+
| 11|                26| 

In [4]:
vassembler = VectorAssembler(inputCols=["MOA", "ROA","distance", "BMI"], outputCol='features')
data = vassembler.transform(data)

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [5]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
#get y_true and y_pred
y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

# Select example rows to display.
predictions.show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+----+-----+----+--------+----+--------------------+------------+--------------------+--------------------+--------------------+----------+--------------+
| ID|Reason for absence|Month of absence|Day of the week|Seasons|Transportation expense|Distance from Residence to Work|Service time|Age|Work load Average/day |Hit target|Disciplinary_failure|Education|Son|Social drinker|Social smoker|Pet|Weight|Height|Body mass index|Absenteeism_time_in_hours| MOA|label| ROA|distance| BMI|            features|indexedLabel|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|
+---+------------------+----------------+---------------+-------+----------------------+--------------

In [6]:
accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

rfModel = model.stages[2]
print(rfModel)  # summary only
print("Random Forest - Test Accuracy = %g" % (accuracy))
print("Random Forest - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Random Forest Model is :\n" + str(confusionmatrix))

print("The precision score for Random Forest Model is: " + str(precision))

print("The recall score for Random Forest Model is: " + str(recall))

RandomForestClassificationModel: uid=RandomForestClassifier_5905c09fd3c9, numTrees=10, numClasses=24, numFeatures=4
Random Forest - Test Accuracy = 0.921053
Random Forest - Test Error = 0.0789474
The Confusion Matrix for Random Forest Model is :
[[61  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 27  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1 23  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  8  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 18  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  8

In [7]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [8]:
# Load the data as a DataFrame.
#data = spark.read.load("winequality-red.csv", format="csv", header=True, delimiter=";")

data = spark.read.format("csv").load("winequality-red.csv", header=True, delimiter=";")

data = data.withColumn("FA", data["fixed acidity"] - 0).withColumn("label", data['quality'] - 0). \
    withColumn("CA", data["citric acid"] - 0). \
    withColumn("RS", data["residual sugar"] - 0). \
    withColumn("A", data["alcohol"] - 0)
data.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+---+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|  FA|label|  CA| RS|  A|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+---+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5| 7.4|  5.0| 0.0|1.9|9.4|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5| 7.8|  5.0| 0.0|2.6|9.8|
|          7.8|            0.76|       0.04|           2.3|    0.092|                 15|                  54|  0.997|3.26|     0.

In [9]:
vassembler = VectorAssembler(inputCols=["FA", "CA", "RS","A"], outputCol='features')

data = vassembler.transform(data)
data.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+---+-------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|  FA|label|  CA| RS|  A|           features|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+---+-------------------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5| 7.4|  5.0| 0.0|1.9|9.4|  [7.4,0.0,1.9,9.4]|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5| 7.8|  5.0| 0.0|2.6|9.8|  [7.8,0.0,2.6,9.8]|
|          7.8|            0.7

In [10]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])


In [11]:
# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
predictions.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+----+--------------------+--------------------+-----------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|  FA|label|  CA| RS|   A|            features|     indexedFeatures|       prediction|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+----+--------------------+--------------------+-----------------+
|           10|            0.26|       0.54|           1.9|    0.083|                 42|                  74|0.99451|2.98|     0.63|   11.8|      8|10.0|  8.0|0.54|1.9|11.8|[10.0,0.54,1.9,11.8]|[10.0,0.54,1.9,11.8]|6.549019607843137|
|           10|            0.35|       0.47|             2| 

In [12]:
# Select (prediction, true label) and compute test error
evaluator_rmse = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)

evaluator_r2 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName= "r2")
r2 = evaluator_r2.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("Coefficient of Determination (R^2) on test data = %g" % r2)

treeModel = model.stages[1]
# summary only
print(treeModel)

Root Mean Squared Error (RMSE) on test data = 0.720801
Coefficient of Determination (R^2) on test data = 0.246783
DecisionTreeRegressionModel: uid=DecisionTreeRegressor_6f11059dfdc3, depth=5, numNodes=59, numFeatures=4


In [13]:
from pyspark.ml.regression import RandomForestRegressor

In [14]:
# Load the data as a DataFrame.
#data = spark.read.load("winequality-red.csv", format="csv", header=True, delimiter=";")

data = spark.read.format("csv").load("winequality-red.csv", header=True, delimiter=";")

data = data.withColumn("FA", data["fixed acidity"] - 0).withColumn("label", data['quality'] - 0). \
    withColumn("CA", data["citric acid"] - 0). \
    withColumn("RS", data["residual sugar"] - 0). \
    withColumn("A", data["alcohol"] - 0)

In [15]:
vassembler = VectorAssembler(inputCols=["FA", "CA", "RS","A"], outputCol='features')

data = vassembler.transform(data)
data.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+---+-------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|  FA|label|  CA| RS|  A|           features|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+---+-------------------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5| 7.4|  5.0| 0.0|1.9|9.4|  [7.4,0.0,1.9,9.4]|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5| 7.8|  5.0| 0.0|2.6|9.8|  [7.8,0.0,2.6,9.8]|
|          7.8|            0.7

In [16]:
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [17]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+----+----+--------------------+--------------------+------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|  FA|label|  CA|  RS|   A|            features|     indexedFeatures|        prediction|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+----+----+--------------------+--------------------+------------------+
|           10|            0.26|       0.54|           1.9|    0.083|                 42|                  74|0.99451|2.98|     0.63|   11.8|      8|10.0|  8.0|0.54| 1.9|11.8|[10.0,0.54,1.9,11.8]|[10.0,0.54,1.9,11.8]| 6.705523479233558|
|           10|            0.29|        0.4|        

In [18]:
# Select (prediction, true label) and compute test error
evaluator_rmse = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)

evaluator_r2 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName= "r2")
r2 = evaluator_r2.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("Coefficient of Determination (R^2) on test data = %g" % r2)

rfModel = model.stages[1]
print(rfModel)  # summary only

Root Mean Squared Error (RMSE) on test data = 0.693439
Coefficient of Determination (R^2) on test data = 0.223466
RandomForestRegressionModel: uid=RandomForestRegressor_1d7430452cf0, numTrees=20, numFeatures=4
